## Trabalho Final - Curso Ciência de Dados

**Autor: Eden de Oliveira Santana**

**1. Análise das Informações e Tratamentos**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def load_csv(path):

  """Função para ler dataframes em formato csv"""

  database = pd.read_csv(path)

  return database


def etl_dataframes(dataframe1, dataframe2, type_how, type_key):

  """JUnção de dataframes segundo tipo  e chave informado pelo usuário"""

  datasets = pd.merge(dataframe1, dataframe2, how=type_how, on=type_key)
  
  return datasets

In [ ]:
df_mv = load_csv('/content/drive/MyDrive/curso-lets-code/ferramentas-programacao/dados02 - movie_lens/movies.csv')
df_rt = load_csv('/content/drive/MyDrive/curso-lets-code/ferramentas-programacao/dados02 - movie_lens/ratings.csv')
df_tags = load_csv('/content/drive/MyDrive/curso-lets-code/ferramentas-programacao/dados02 - movie_lens/tags.csv')

In [ ]:
df_mv = df_mv.astype({"movieId": str})
df_rt = df_rt.astype({"movieId": str, "userId": str})
df_tags = df_tags.astype({"movieId": str, "userId": str})

In [ ]:
df_mv.head(), df_rt.head(), df_tags.head()

(  movieId                               title  \
 0       1                    Toy Story (1995)   
 1       2                      Jumanji (1995)   
 2       3             Grumpier Old Men (1995)   
 3       4            Waiting to Exhale (1995)   
 4       5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  ,
   userId movieId  rating  timestamp
 0      1       1     4.0  964982703
 1      1       3     4.0  964981247
 2      1       6     4.0  964982224
 3      1      47     5.0  964983815
 4      1      50     5.0  964982931,
   userId movieId              tag   timestamp
 0      2   60756            funny  1445714994
 1      2   60756  Highly quotable  1445714996
 2      2   60756     will ferrel

In [ ]:
df_mv.shape, df_rt.shape, df_tags.shape

((9742, 3), (100836, 4), (3683, 4))

In [ ]:
#Separação das colunas gêneros
df_mv_desagreg=df_mv.assign(genres=df_mv.genres.str.split('|')).explode('genres')

In [ ]:
df_rt.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
df_join = etl_dataframes(df_mv_desagreg, df_rt, "outer", "movieId")
df_desagreg_tags = etl_dataframes(df_join, df_tags, "outer", "movieId")

In [ ]:
df_join.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure,1,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure,5,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure,7,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure,15,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure,17,4.5,1.305696e+09


In [ ]:
df_desagreg_tags.head()

,movieId,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y
0,1,Toy Story (1995),Adventure,1,4.0,964982703.0,336,pixar,1.139046e+09
1,1,Toy Story (1995),Adventure,1,4.0,964982703.0,474,pixar,1.137207e+09
2,1,Toy Story (1995),Adventure,1,4.0,964982703.0,567,fun,1.525286e+09
3,1,Toy Story (1995),Adventure,5,4.0,847434962.0,336,pixar,1.139046e+09
4,1,Toy Story (1995),Adventure,5,4.0,847434962.0,474,pixar,1.137207e+09


In [ ]:
df_desagreg_tags.drop(['timestamp_x', 'timestamp_y'], axis=1, inplace=True)

In [ ]:
df_desagreg_tags.shape

(905196, 7)

In [ ]:
df_desagreg_tags.dtypes

movieId      object
title        object
genres       object
userId_x     object
rating      float64
userId_y     object
tag          object
dtype: object

In [ ]:
#df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)

**1.1. Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?**

In [ ]:
#df_join.title.unique().tolist()

**Após os tratamentos filtramos o filme que possui o gênero Matriz**

In [ ]:
df_matrix = df_desagreg_tags.loc[df_desagreg_tags['tag'].isin(["Matrix"])]

In [ ]:
 df_matrix.tail()

,movieId,title,genres,userId_x,rating,userId_y,tag
741091,27660,"Animatrix, The (2003)",Sci-Fi,522,3.5,62,Matrix
741097,27660,"Animatrix, The (2003)",Sci-Fi,525,3.0,62,Matrix
741103,27660,"Animatrix, The (2003)",Sci-Fi,534,4.5,62,Matrix
741109,27660,"Animatrix, The (2003)",Sci-Fi,560,4.0,62,Matrix
741115,27660,"Animatrix, The (2003)",Sci-Fi,599,2.5,62,Matrix


In [ ]:
df_matrix.groupby(['title'])['genres'].agg('count').reset_index()

,title,genres
0,"Animatrix, The (2003)",80


In [ ]:
df_matrix.groupby(['genres'])['userId_x'].agg('count').reset_index()

,genres,userId_x
0,Action,20
1,Animation,20
2,Drama,20
3,Sci-Fi,20


**Possui 80 gêneros, sendo eles Action, Animation, Drama e Sci-Fi.**

**1.2. Quantos gêneros cada filme tem, em média?**

In [ ]:
df_agreg_genres = df_desagreg_tags.groupby(['genres'])['userId_x'].agg('count').reset_index().sort_values(by=['userId_x'], ascending=False)

In [ ]:
df_agreg_genres[["userId_x"]].mean()

userId_x    45257.65
dtype: float64

**Em média cada filme tem 45257.65 gêneros**

**1.3. Quais são os cinco filmes com maior número de gêneros?**

In [ ]:
df_agreg_genres.head()

,genres,userId_x
8,Drama,167783
17,Thriller,130834
5,Comedy,119180
6,Crime,110712
1,Action,86976


**1.4.Quantos usuários deram nota para o filme "The Matrix"?**

In [ ]:
df_matrix.groupby(['title'])['rating'].agg('count').reset_index()

,title,rating
0,"Animatrix, The (2003)",80


**1.5. Quantos usuários, em média, deram nota para cada filme?**

In [ ]:
#df_desagreg_tags.groupby("title").agg({"userId_x": [np.count_nonzero]}).reset_index().sort_values(by=['title'], ascending=False)
df_agreg_rating_movies = df_desagreg_tags.groupby(['title','rating'])['userId_x'].agg('count').reset_index().sort_values(by=['userId_x'], ascending=False)
df_agreg_rating_movies.head(10)

,title,rating,userId_x
21524,Pulp Fiction (1994),5.0,89052
21522,Pulp Fiction (1994),4.0,55748
21523,Pulp Fiction (1994),4.5,31856
21520,Pulp Fiction (1994),3.0,22444
9442,Fight Club (1999),5.0,17496
9441,Fight Club (1999),4.5,10584
9440,Fight Club (1999),4.0,10584
21521,Pulp Fiction (1994),3.5,8688
25356,Star Wars: Episode IV - A New Hope (1977),5.0,8112
13480,Inception (2010),5.0,7462


In [ ]:
df_agreg_rating_movies[["userId_x"]].mean()

userId_x    29.762043
dtype: float64

**Em média 29.76 usuários deram nota para cada filme.**

In [ ]:
(df_desagreg_tags['rating'] == 0).all()

False

**1.6. Quais são os cinco filmes para os quais mais usuários deram nota?**

In [ ]:
df_agreg_rating_movies.head()

,title,rating,userId_x
21524,Pulp Fiction (1994),5.0,89052
21522,Pulp Fiction (1994),4.0,55748
21523,Pulp Fiction (1994),4.5,31856
21520,Pulp Fiction (1994),3.0,22444
9442,Fight Club (1999),5.0,17496


**1.7. Qual o rating médio do filme "The Matrix"?**

In [ ]:
df_matrix[["rating"]].mean()

rating    3.7
dtype: float64

**1.8. Cada filme tem um rating médio. Qual a média desses ratings, por gênero?**

In [ ]:
df_agreg_genres_rating = df_desagreg_tags.groupby(['genres', 'rating'])['userId_x'].agg('count').reset_index().sort_values(by=['userId_x'], ascending=False)

#filtro apenas para filmes que possuem gênero listado
df_filter_genres_rating = df_agreg_genres_rating[~df_agreg_genres_rating['genres'].isin(["(no genres listed)"])]
print(df_filter_genres_rating.head())
print()
print(df_filter_genres_rating[["rating"]].mean())

       genres  rating  userId_x
88      Drama     5.0     47131
86      Drama     4.0     46027
178  Thriller     5.0     38507
68      Crime     5.0     36215
176  Thriller     4.0     34428

rating    2.75
dtype: float64


**Após aplicar o filtro removendo a tag (no genres listed), o rating médio por gênero é de 2.75**

**1.9. Cada filme tem um rating médio. Qual a média desses ratings, por gênero?**

In [ ]:
df_filter_genres_rating[["userId_x"]].count()

userId_x    190
dtype: int64